<a href="https://colab.research.google.com/github/philippengani/shoppy/blob/main/shoppy_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's setup Spark on your Colab environment. Run the cell below!


In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!pip install findspark

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [2]:
import pandas as pd
from pyspark import SparkConf, SparkContext, sql
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import findspark
findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [4]:
spark

In [5]:
# Creating the database connection to the shoppy mysql database

jdbcHostname = "novobyte.org"
jdbcDatabase = "jsvx9284_shpy"
username = 'jsvx9284_ndserge'
password = 'DX%A818lF.+]'
jdbcPort = 3306
jdbcUrl = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}".format(jdbcHostname, jdbcPort, jdbcDatabase, username, password)



In [6]:
# Importing the shoppy ratings data and product data

#shoppy_ratings_df = spark.read.jdbc(url=jdbcUrl, table="shoppy_ratings")
shoppy_ratings_df = spark.read.format("jdbc")\
                        .option("url", "jdbc:mysql://novobyte.org/jsvx9284_shpy?useLegacyDatetimeCode=false&serverTimezone=UTC")\
                        .option("driver", "com.mysql.jdbc.Driver")\
                        .option("dbtable", "shoppy_ratings")\
                        .option("user", "jsvx9284_ndserge")\
                        .option("password", "DX%A818lF.+]")\
                        .option("customSchema", "rating_id INT, rating_value INT, user_id INT, reference_id INT , type STRING")\
                        .load()

shoppy_products_df = spark.read.format("jdbc")\
                        .option("url", "jdbc:mysql://novobyte.org/jsvx9284_shpy?useLegacyDatetimeCode=false&serverTimezone=UTC")\
                        .option("driver", "com.mysql.jdbc.Driver")\
                        .option("query", "select product_id, product_name_en,actual_price from shoppy_products")\
                        .option("user", "jsvx9284_ndserge")\
                        .option("password", "DX%A818lF.+]")\
                        .option("customSchema", "product_id INT, product_name_en STRING, actual_price DECIMAL(38, 0)") \
                        .load()

In [7]:
# Caching both dataframes because they will be used many times
# It is better to avoid getting them all the time

shoppy_ratings_df.cache()
shoppy_products_df.cache()

print(" Shoppy ratings table : ")
shoppy_ratings_df.show(10)

print(" Shoppy products table : ")
shoppy_products_df.show(10, truncate=False)

 Shoppy ratings table : 
+---------+------------+-------+------------+-------+
|rating_id|rating_value|user_id|reference_id|   type|
+---------+------------+-------+------------+-------+
|        8|           4|     14|           4|   SHOP|
|        9|           3|     14|           1|   SHOP|
|       10|           5|     14|          35|PRODUCT|
|       11|           3|     17|          56|PRODUCT|
|       12|           3|     17|           7|   SHOP|
|       13|           2|     17|           4|   SHOP|
|       14|           3|     14|          64|PRODUCT|
|       15|           5|     41|         100|PRODUCT|
|       16|           4|     41|          64|PRODUCT|
|       17|           2|     14|          12|   SHOP|
+---------+------------+-------+------------+-------+
only showing top 10 rows

 Shoppy products table : 
+----------+-----------------------------+------------+
|product_id|product_name_en              |actual_price|
+----------+-----------------------------+------------+

# Visualization Tools

In [8]:
pip install plotly --upgrade 

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (5.0.0)


In [9]:
df = shoppy_ratings_df.toPandas()
df_product = df [df["type"]=='PRODUCT']
df_product_2= pd.DataFrame(columns=["id_product"],data=df_product.reference_id.unique())
df_product_2["rating_value"]=(df_product.groupby("reference_id")["rating_value"].mean().to_list())

In [10]:
df_product_2

,id_product,rating_value
0,35,5.0
1,56,3.0
2,64,3.5
3,100,4.0
4,372,4.0
...,...,...
118,1532,4.0
119,1373,4.0
120,1539,4.0
121,1527,4.0


# Most Popular Rating (average)

In [11]:
import plotly.express as px
fig = px.scatter(df_product_2, x="id_product", y="rating_value", color="rating_value", size="rating_value")
fig.show(renderer="colab")

In [12]:
display(shoppy_ratings_df)

DataFrame[rating_id: int, rating_value: int, user_id: int, reference_id: int, type: string]

In [13]:
shoppy_ratings_df.groupBy('reference_id').avg().head(10)

[Row(reference_id=1507, avg(rating_id)=154.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=1507.0),
 Row(reference_id=1303, avg(rating_id)=94.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=1303.0),
 Row(reference_id=1322, avg(rating_id)=123.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=1322.0),
 Row(reference_id=53, avg(rating_id)=60.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=53.0),
 Row(reference_id=1404, avg(rating_id)=143.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=1404.0),
 Row(reference_id=1378, avg(rating_id)=135.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=1378.0),
 Row(reference_id=81, avg(rating_id)=152.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(reference_id)=81.0),
 Row(reference_id=436, avg(rating_id)=55.0, avg(rating_value)=4.0, avg(user_id)=47.0, avg(reference_id)=436.0),
 Row(reference_id=1031, avg(rating_id)=132.0, avg(rating_value)=4.0, avg(user_id)=26.0, avg(r

In [14]:
#Get only the products
products_df = shoppy_ratings_df.where(shoppy_ratings_df.type =="PRODUCT")

In [15]:
products_df.show(10)


+---------+------------+-------+------------+-------+
|rating_id|rating_value|user_id|reference_id|   type|
+---------+------------+-------+------------+-------+
|       10|           5|     14|          35|PRODUCT|
|       11|           3|     17|          56|PRODUCT|
|       14|           3|     14|          64|PRODUCT|
|       15|           5|     41|         100|PRODUCT|
|       16|           4|     41|          64|PRODUCT|
|       18|           4|     14|         372|PRODUCT|
|       19|           0|     68|         401|PRODUCT|
|       25|           3|     26|         415|PRODUCT|
|       26|           4|     68|         440|PRODUCT|
|       27|           3|     68|         406|PRODUCT|
+---------+------------+-------+------------+-------+
only showing top 10 rows



In [16]:
# Compute the average rating value and the count for each products
from pyspark.sql import functions as F

grouped_prod_df = products_df.groupBy('reference_id').agg(F.count(products_df.rating_value).alias("count"), F.mean(products_df.rating_value).alias("average_rating"))


In [17]:
grouped_prod_df.filter("count >= 2").show(10,truncate=False)

+------------+-----+--------------+
|reference_id|count|average_rating|
+------------+-----+--------------+
|436         |2    |4.0           |
|412         |2    |3.5           |
|64          |2    |3.5           |
|437         |2    |4.0           |
|533         |2    |4.5           |
|933         |2    |1.5           |
+------------+-----+--------------+



In [18]:
# Get the grouped product df with the name of the products

grouped_prod_names = grouped_prod_df.join(shoppy_products_df,grouped_prod_df['reference_id'] == shoppy_products_df['product_id'])
grouped_prod_names.show(10)

+------------+-----+--------------+----------+--------------------+------------+
|reference_id|count|average_rating|product_id|     product_name_en|actual_price|
+------------+-----+--------------+----------+--------------------+------------+
|          35|    1|           5.0|        35| Traditional Sandals|        2000|
|          56|    1|           3.0|        56|Samsung Galaxy no...|       98000|
|          64|    2|           3.5|        64|           IPhone 5s|       80000|
|          97|    1|           4.0|        97|Customized ballerina|        3500|
|          98|    1|           4.0|        98|Flip flop, earing...|        8000|
|          99|    1|           4.0|        99| Customized hand bag|       10000|
|         100|    1|           5.0|       100|Bangles and Barrette|        2000|
|         173|    1|           3.0|       173|Polo By Ralph Lau...|        6000|
|         216|    1|           3.0|       216|      Earpeace (AKJ)|        1000|
|         260|    1|        

In [25]:
# Get select the required fields from the above table
grouped_prod_names = grouped_prod_names.select("average_rating", "product_name_en", "count", "product_id").sort('average_rating', ascending = False )

print(" The top 10 most popular products")
grouped_prod_names.show(10)




 The top 10 most popular products
+--------------+--------------------+-----+----------+
|average_rating|     product_name_en|count|product_id|
+--------------+--------------------+-----+----------+
|           5.0|deer tree jewelry...|    1|      1314|
|           5.0|Bangles and Barrette|    1|       100|
|           5.0| Traditional Sandals|    1|        35|
|           4.5|Samsung galaxy s1...|    2|       533|
|           4.0|  Ankara Flower pots|    1|       263|
|           4.0|     Alexander Shoes|    1|       348|
|           4.0|African fabric ta...|    1|       266|
|           4.0|Flip flop, earing...|    1|        98|
|           4.0|          Nike Shoes|    1|       355|
|           4.0|      Tecno camon cx|    1|       407|
+--------------+--------------------+-----+----------+
only showing top 10 rows



In [41]:
import plotly.express as px
import plotly.graph_objects as go

grouped_prod_names_pd =grouped_prod_names.toPandas()

# Get the 20 products with the largest rating

grouped_prod_names_pd = grouped_prod_names_pd.nlargest(20, 'average_rating')

'''
fig = px.bar(grouped_prod_names_pd, x='product_id', y='average_rating')
fig.show()
'''

# Use textposition='auto' for direct text
grouped_prod_names_pd['product_id'] = grouped_prod_names_pd['product_id'].astype(str)
fig = go.Figure(data=[go.Bar(
            x=grouped_prod_names_pd.product_id, y=grouped_prod_names_pd.average_rating,
            text=grouped_prod_names_pd.product_name_en,
            textposition='auto',
        )])

fig.show()

In [44]:
print(" The list of the top 20 products :")
grouped_prod_names_pd

 The list of the top 20 products :


,average_rating,product_name_en,count,product_id
0,5.0,Traditional Sandals,1,35
1,5.0,Bangles and Barrette,1,100
2,5.0,deer tree jewelry stands,1,1314
3,4.5,Samsung galaxy s10 duos 128go,2,533
4,4.0,Customized ballerina,1,97
5,4.0,"Flip flop, earing and purse",1,98
6,4.0,Customized hand bag,1,99
7,4.0,Wall hangings,1,260
8,4.0,Wall hangings,1,262
9,4.0,Ankara Flower pots,1,263


# Products mapping